In [154]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

import data and remove NaNs for features:

In [155]:
df = pd.read_csv('index.csv')
df.insert(0, "Date", pd.to_datetime(df[["Year", "Month", "Day"]]))      # create datetime column
df.set_index(["Date"], inplace=True)                                    # set index to date, easy to handle 
df.drop(columns=["Year", "Month", "Day"], inplace=True)                 # deop redundant columns

In [156]:
df.drop(columns=["Federal Funds Target Rate", "Federal Funds Upper Target", "Federal Funds Lower Target"], inplace=True)        # these columns will not be used

In [157]:
targ_df = pd.DataFrame(df["Real GDP (Percent Change)"])     # create target and drop the NaNs to get to quarterly values
targ_df.head(5)

,Real GDP (Percent Change)
Date,
1954-07-01,4.6
1954-08-01,NaN
1954-09-01,NaN
1954-10-01,8.0
1954-11-01,NaN


In [158]:
df["Real GDP (Percent Change)"] = df["Real GDP (Percent Change)"].interpolate()       # interpolate the target so we can create quarterly lags

In [159]:
df.head(10)

,Effective Federal Funds Rate,Real GDP (Percent Change),Unemployment Rate,Inflation Rate
Date,,,,
1954-07-01,0.80,4.600000,5.8,NaN
1954-08-01,1.22,5.733333,6.0,NaN
1954-09-01,1.06,6.866667,6.1,NaN
1954-10-01,0.85,8.000000,5.7,NaN
1954-11-01,0.83,9.300000,5.3,NaN
1954-12-01,1.28,10.600000,5.0,NaN
1955-01-01,1.39,11.900000,4.9,NaN
1955-02-01,1.29,10.166667,4.7,NaN
1955-03-01,1.35,8.433333,4.6,NaN


In [160]:
# determine date when inflation rate begins to ve recorded 
for i,j in enumerate(df["Inflation Rate"]):
    if not np.isnan(j):
        print(df.iloc[i])
        break                   

Effective Federal Funds Rate     2.72
Real GDP (Percent Change)      -10.00
Unemployment Rate                5.80
Inflation Rate                   3.20
Name: 1958-01-01 00:00:00, dtype: float64


In [161]:
df.drop(df.loc["1954-07-01":"1958-01-01"].index, inplace=True)      # drop the rows with no inflation rate


In [162]:
df.drop(df.loc["2016-11-01":].index, inplace=True) # drop as there is no target for these dates

In [163]:
df.tail(10)

,Effective Federal Funds Rate,Real GDP (Percent Change),Unemployment Rate,Inflation Rate
Date,,,,
2016-01-01,0.34,0.800000,4.9,2.2
2016-02-01,0.38,1.000000,4.9,2.3
2016-03-01,0.36,1.200000,5.0,2.2
2016-04-01,0.37,1.400000,5.0,2.1
2016-05-01,0.37,2.100000,4.7,2.2
2016-06-01,0.38,2.800000,4.9,2.2
2016-07-01,0.39,3.500000,4.9,2.2
2016-08-01,0.40,2.966667,4.9,2.3
2016-09-01,0.40,2.433333,4.9,2.2


In [164]:
# remove mid-month entries as these were wgen target rates were chanmged, theses only contain NaNs for our feqture vaeriables 
idx_mm = [i for i,j in enumerate(df.index.day) if j != 1]
df.drop(df.iloc[idx_mm].index, inplace=True) 


In [165]:
df[["Unemployment Rate", "Effective Federal Funds Rate", "Inflation Rate"]].isna().sum()        # check if all are removed

Unemployment Rate               0
Effective Federal Funds Rate    0
Inflation Rate                  0
dtype: int64

Feature engineering - Create Lags:

In [166]:
lags = [1,2,3,4,5]        # introduce lag features 
predictors = ["Effective Federal Funds Rate", "Inflation Rate", "Unemployment Rate", "Real GDP (Percent Change)"]  # include target to create its own lags 

# loop and dynamically name lags
for col in predictors:
    for L in lags:
        df[f"{col} lag{L}"] = df[col].shift(L)
df.head(8)

,Effective Federal Funds Rate,Real GDP (Percent Change),Unemployment Rate,Inflation Rate,Effective Federal Funds Rate lag1,Effective Federal Funds Rate lag2,Effective Federal Funds Rate lag3,Effective Federal Funds Rate lag4,Effective Federal Funds Rate lag5,Inflation Rate lag1,...,Unemployment Rate lag1,Unemployment Rate lag2,Unemployment Rate lag3,Unemployment Rate lag4,Unemployment Rate lag5,Real GDP (Percent Change) lag1,Real GDP (Percent Change) lag2,Real GDP (Percent Change) lag3,Real GDP (Percent Change) lag4,Real GDP (Percent Change) lag5
Date,,,,,,,,,,,,,,,,,,,,,
1958-02-01,1.67,-5.800000,6.4,3.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1958-03-01,1.20,-1.600000,6.7,2.8,1.67,NaN,NaN,NaN,NaN,3.2,...,6.4,NaN,NaN,NaN,NaN,-5.800000,NaN,NaN,NaN,NaN
1958-04-01,1.26,2.600000,7.4,2.4,1.20,1.67,NaN,NaN,NaN,2.8,...,6.7,6.4,NaN,NaN,NaN,-1.600000,-5.800000,NaN,NaN,NaN
1958-05-01,0.63,4.933333,7.4,2.4,1.26,1.20,1.67,NaN,NaN,2.4,...,7.4,6.7,6.4,NaN,NaN,2.600000,-1.600000,-5.800000,NaN,NaN
1958-06-01,0.93,7.266667,7.3,2.1,0.63,1.26,1.20,1.67,NaN,2.4,...,7.4,7.4,6.7,6.4,NaN,4.933333,2.600000,-1.600000,-5.800000,NaN
1958-07-01,0.68,9.600000,7.5,2.4,0.93,0.63,1.26,1.20,1.67,2.1,...,7.3,7.4,7.4,6.7,6.4,7.266667,4.933333,2.600000,-1.600000,-5.8
1958-08-01,1.53,9.633333,7.4,2.1,0.68,0.93,0.63,1.26,1.20,2.4,...,7.5,7.3,7.4,7.4,6.7,9.600000,7.266667,4.933333,2.600000,-1.6
1958-09-01,1.76,9.666667,7.1,1.7,1.53,0.68,0.93,0.63,1.26,2.1,...,7.4,7.5,7.3,7.4,7.4,9.633333,9.600000,7.266667,4.933333,2.6


In [167]:
df.drop(df.iloc[0:6].index, inplace=True)       # drop the first 5 as these contains NaNs due to way lags were created
df.head(10)

,Effective Federal Funds Rate,Real GDP (Percent Change),Unemployment Rate,Inflation Rate,Effective Federal Funds Rate lag1,Effective Federal Funds Rate lag2,Effective Federal Funds Rate lag3,Effective Federal Funds Rate lag4,Effective Federal Funds Rate lag5,Inflation Rate lag1,...,Unemployment Rate lag1,Unemployment Rate lag2,Unemployment Rate lag3,Unemployment Rate lag4,Unemployment Rate lag5,Real GDP (Percent Change) lag1,Real GDP (Percent Change) lag2,Real GDP (Percent Change) lag3,Real GDP (Percent Change) lag4,Real GDP (Percent Change) lag5
Date,,,,,,,,,,,,,,,,,,,,,
1958-08-01,1.53,9.633333,7.4,2.1,0.68,0.93,0.63,1.26,1.20,2.4,...,7.5,7.3,7.4,7.4,6.7,9.600000,7.266667,4.933333,2.600000,-1.600000
1958-09-01,1.76,9.666667,7.1,1.7,1.53,0.68,0.93,0.63,1.26,2.1,...,7.4,7.5,7.3,7.4,7.4,9.633333,9.600000,7.266667,4.933333,2.600000
1958-10-01,1.80,9.700000,6.7,1.7,1.76,1.53,0.68,0.93,0.63,1.7,...,7.1,7.4,7.5,7.3,7.4,9.666667,9.633333,9.600000,7.266667,4.933333
1958-11-01,2.27,9.033333,6.2,1.7,1.80,1.76,1.53,0.68,0.93,1.7,...,6.7,7.1,7.4,7.5,7.3,9.700000,9.666667,9.633333,9.600000,7.266667
1958-12-01,2.42,8.366667,6.2,1.7,2.27,1.80,1.76,1.53,0.68,1.7,...,6.2,6.7,7.1,7.4,7.5,9.033333,9.700000,9.666667,9.633333,9.600000
1959-01-01,2.48,7.700000,6.0,1.7,2.42,2.27,1.80,1.76,1.53,1.7,...,6.2,6.2,6.7,7.1,7.4,8.366667,9.033333,9.700000,9.666667,9.633333
1959-02-01,2.43,8.500000,5.9,1.7,2.48,2.42,2.27,1.80,1.76,1.7,...,6.0,6.2,6.2,6.7,7.1,7.700000,8.366667,9.033333,9.700000,9.666667
1959-03-01,2.80,9.300000,5.6,1.7,2.43,2.48,2.42,2.27,1.80,1.7,...,5.9,6.0,6.2,6.2,6.7,8.500000,7.700000,8.366667,9.033333,9.700000
1959-04-01,2.96,10.100000,5.2,1.7,2.80,2.43,2.48,2.42,2.27,1.7,...,5.6,5.9,6.0,6.2,6.2,9.300000,8.500000,7.700000,8.366667,9.033333


In [168]:
df.drop(columns="Real GDP (Percent Change)", inplace=True)      # drop the target
df.isna().sum()

Effective Federal Funds Rate         0
Unemployment Rate                    0
Inflation Rate                       0
Effective Federal Funds Rate lag1    0
Effective Federal Funds Rate lag2    0
Effective Federal Funds Rate lag3    0
Effective Federal Funds Rate lag4    0
Effective Federal Funds Rate lag5    0
Inflation Rate lag1                  0
Inflation Rate lag2                  0
Inflation Rate lag3                  0
Inflation Rate lag4                  0
Inflation Rate lag5                  0
Unemployment Rate lag1               0
Unemployment Rate lag2               0
Unemployment Rate lag3               0
Unemployment Rate lag4               0
Unemployment Rate lag5               0
Real GDP (Percent Change) lag1       0
Real GDP (Percent Change) lag2       0
Real GDP (Percent Change) lag3       0
Real GDP (Percent Change) lag4       0
Real GDP (Percent Change) lag5       0
dtype: int64

Now need to reduce to quartely evaluation: 

In [181]:
df.tail(5)

,Effective Federal Funds Rate,Unemployment Rate,Inflation Rate,Effective Federal Funds Rate lag1,Effective Federal Funds Rate lag2,Effective Federal Funds Rate lag3,Effective Federal Funds Rate lag4,Effective Federal Funds Rate lag5,Inflation Rate lag1,Inflation Rate lag2,...,Unemployment Rate lag1,Unemployment Rate lag2,Unemployment Rate lag3,Unemployment Rate lag4,Unemployment Rate lag5,Real GDP (Percent Change) lag1,Real GDP (Percent Change) lag2,Real GDP (Percent Change) lag3,Real GDP (Percent Change) lag4,Real GDP (Percent Change) lag5
Date,,,,,,,,,,,,,,,,,,,,,
2015-10-01,0.12,5.0,1.9,0.14,0.14,0.13,0.13,0.12,1.9,1.8,...,5.0,5.1,5.2,5.3,5.5,1.266667,1.633333,2.0,2.200000,2.400000
2016-01-01,0.34,4.9,2.2,0.24,0.12,0.12,0.14,0.14,2.1,2.0,...,5.0,5.0,5.0,5.0,5.1,0.850000,0.875000,0.9,1.266667,1.633333
2016-04-01,0.37,5.0,2.1,0.36,0.38,0.34,0.24,0.12,2.2,2.3,...,5.0,4.9,4.9,5.0,5.0,1.200000,1.000000,0.8,0.850000,0.875000
2016-07-01,0.39,4.9,2.2,0.38,0.37,0.37,0.36,0.38,2.2,2.2,...,4.9,4.7,5.0,5.0,4.9,2.800000,2.100000,1.4,1.200000,1.000000
2016-10-01,0.40,4.8,2.1,0.40,0.40,0.39,0.38,0.37,2.2,2.3,...,4.9,4.9,4.9,4.9,4.7,2.433333,2.966667,3.5,2.800000,2.100000


In [182]:
targ_df = targ_df.loc["1958-08-01":"2016-10-01"].dropna()
targ_df.head(5)


,Real GDP (Percent Change)
Date,
1958-10-01,9.7
1959-01-01,7.7
1959-04-01,10.1
1959-07-01,-0.8
1959-10-01,1.6


In [183]:
df = df.loc[targ_df.index]
df.head(5)

,Effective Federal Funds Rate,Unemployment Rate,Inflation Rate,Effective Federal Funds Rate lag1,Effective Federal Funds Rate lag2,Effective Federal Funds Rate lag3,Effective Federal Funds Rate lag4,Effective Federal Funds Rate lag5,Inflation Rate lag1,Inflation Rate lag2,...,Unemployment Rate lag1,Unemployment Rate lag2,Unemployment Rate lag3,Unemployment Rate lag4,Unemployment Rate lag5,Real GDP (Percent Change) lag1,Real GDP (Percent Change) lag2,Real GDP (Percent Change) lag3,Real GDP (Percent Change) lag4,Real GDP (Percent Change) lag5
Date,,,,,,,,,,,,,,,,,,,,,
1958-10-01,1.80,6.7,1.7,1.76,1.53,0.68,0.93,0.63,1.7,2.1,...,7.1,7.4,7.5,7.3,7.4,9.666667,9.633333e+00,9.6,7.266667,4.933333
1959-01-01,2.48,6.0,1.7,2.42,2.27,1.80,1.76,1.53,1.7,1.7,...,6.2,6.2,6.7,7.1,7.4,8.366667,9.033333e+00,9.7,9.666667,9.633333
1959-04-01,2.96,5.2,1.7,2.80,2.43,2.48,2.42,2.27,1.7,1.7,...,5.6,5.9,6.0,6.2,6.2,9.300000,8.500000e+00,7.7,8.366667,9.033333
1959-07-01,3.47,5.1,2.0,3.39,2.90,2.96,2.80,2.43,2.0,2.0,...,5.0,5.1,5.2,5.6,5.9,2.833333,6.466667e+00,10.1,9.300000,8.500000
1959-10-01,3.98,5.7,2.7,3.76,3.50,3.47,3.39,2.90,2.4,2.0,...,5.5,5.2,5.1,5.0,5.1,0.800000,1.110223e-16,-0.8,2.833333,6.466667
